# Simple Spam Email Filtering

In [1]:
import nltk
import os
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
for (path,dirs,files) in os.walk('enron1/spam'):
    file_name = files[1:]
for (path,dirs,files) in os.walk('enron1/ham'):
    ham_file = files[1:]

Created a list of words used in all 1500 spam emails (except stopwords and punctuations). Also created a dictionary that indicates a set of words in each spam email.

In [2]:
filtered = []
file_dict = {}
stopwords = set(stopwords.words('english'))
for i in range(len(file_name)):
    file = open('enron1/spam/'+file_name[i],encoding='utf-8',errors='ignore').read()
    tokens = nltk.word_tokenize(file)
    single_doc=set([])
    for w in tokens:
        if w not in stopwords:
            if w.isalpha():
                single_doc.add(w.lower())
                filtered.append(w.lower())
    file_dict[file_name[i]] = single_doc
#print(len(file_dict))

Created a list of words used in all 3672 ham emails (except stopwords and punctuations). Also created a dictionary that indicates a set of words in each ham email.

In [3]:
ham = []
ham_dict={}
for i in range(len(ham_file)):
    file = open('enron1/ham/'+ham_file[i],encoding='utf-8',errors='ignore').read()
    tokens = nltk.word_tokenize(file)
    single_doc=set([])
    for w in tokens:
        if w not in stopwords:
            if w.isalpha():
                single_doc.add(w.lower())
                ham.append(w.lower())
    ham_dict[ham_file[i]] = single_doc
#print(len(ham_dict))

Extracted the most common 1000 words from spam and ham list to create the spam dictionary and ham dictionary. 
After that, P(x_i|y=spam) and P(x_i|y=ham) are calculated based on the frequency of every word in each dictionary, respectively.

In [4]:
dict1=[]
ham_freq = {}
hamfreq = nltk.FreqDist(ham)
for i in hamfreq.most_common(1000):
    dict1.append(i[0])
    ham_freq[i[0]]=0

for key in ham_dict:
    for i in ham_dict[key]:
        if i in dict1:
            ham_freq[i]+=1
for key in ham_freq:
    ham_freq[key] = ham_freq[key]/3672
#print(ham_freq)

In [5]:
spam_freq = {}
dict=[]
freq = nltk.FreqDist(filtered)

for i in freq.most_common(1000):
    dict.append(i[0])
    spam_freq[i[0]]=0
for key in file_dict:
    for i in file_dict[key]:
        if i in dict:
            spam_freq[i] +=1
for key in spam_freq:
    spam_freq[key] = spam_freq[key]/1500
#print(spam_freq)

In [6]:
p_spam= 0.25
p_ham = 0.75

def spam_ind(file):
    input = []
    spam_prob = []
    token = nltk.word_tokenize(open(file).read())     
    for w in token:
        if w not in stopwords:
            if w.isalpha():
                input.append(w.lower())
    #print(input)
    for i in spam_freq:
        if i in input:
            #print(i)
            spam_prob.append(spam_freq[i])
        else:
            #print(i)
            spam_prob.append(1-spam_freq[i])
    return np.asarray(spam_prob)

def ham_ind(file):
    input = []
    ham_prob = []
    token = nltk.word_tokenize(open(file).read())     
    for w in token:
        if w not in stopwords:
            if w.isalpha():
                input.append(w.lower())
    for i in ham_freq:
        if i in input:
            ham_prob.append(ham_freq[i])
        else:
            ham_prob.append(1-ham_freq[i])
    return np.asarray(ham_prob)
spam_prob = spam_ind('enron1/spam/0008.2003-12-18.GP.spam.txt')
ham_prob = ham_ind('enron1/spam/0008.2003-12-18.GP.spam.txt')
#print(ham_prob)
#print(spam_prob)
print(np.log(p_spam*np.prod(spam_prob)))
print(np.log(p_ham*np.prod(ham_prob)))
print(p_spam*np.prod(spam_prob)+p_ham*np.prod(ham_prob))
final_dec = (p_spam*np.prod(spam_prob))/((p_spam*np.prod(spam_prob))+(p_ham*np.prod(ham_prob)))
print(final_dec)

-129.89316673560555
-85.89972364233488
4.945657364560455e-38
7.832320032882479e-20
